<a href="https://colab.research.google.com/github/amitarav16/deep_learning/blob/master/transfer_learning(catsvsdogs).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#imports
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model

In [2]:
#downloading the cats vs dogs dataset
!wget "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip" -O "/tmp/cats_and_dogs_filtered.zip"

--2019-09-24 11:27:08--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.128, 2607:f8b0:400c:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   183MB/s    in 0.4s    

2019-09-24 11:27:09 (183 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [0]:
#unzipping
import zipfile
main_file = '/tmp/cats_and_dogs_filtered.zip'
zipref = zipfile.ZipFile(main_file,'r')
zipref.extractall()
zipref.close()

In [4]:
#pointing the directories
main_dir = '/content/cats_and_dogs_filtered'
training_dir = os.path.join(main_dir,'train')
validation_dir = os.path.join(main_dir,'validation')
train_cats = os.path.join(training_dir,'cats')
train_dogs = os.path.join(training_dir,'dogs')
validation_cats = os.path.join(validation_dir,'cats')
validation_dogs = os.path.join(validation_dir,'dogs')
print(len(os.listdir(train_cats)))
print(len(os.listdir(train_dogs)))
print(len(os.listdir(validation_cats)))
print(len(os.listdir(validation_dogs)))

1000
1000
500
500


In [5]:
#preprocessing
#image augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=40,width_shift_range=0.2,height_shift_range=0.2,zoom_range=0.2,shear_range=0.2,horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(training_dir,batch_size=20,class_mode='binary',target_size=(150,150))
validation_generator = train_datagen.flow_from_directory(validation_dir,batch_size=20,class_mode='binary',target_size=(150,150))


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [8]:
#loading the pre trained model
#transfer learning
base_model = tf.keras.applications.MobileNetV2(input_shape=(150,150,3),include_top=False,weights='imagenet')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


9412608/9406464 [==============================] - 0s 0us/step


In [10]:
#locking the model
base_model.trainable = False
base_model.summary()
#adding some layers on top of base_model
model = tf.keras.models.Sequential([base_model,
                                    tf.keras.layers.GlobalAveragePooling2D(),
                                    tf.keras.layers.Dense(1,activation='sigmoid')])

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 151, 151, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 75, 75, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 75, 75, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

In [11]:
#compiling the model
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(lr=0.0001),loss='binary_crossentropy',metrics=['acc'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [12]:
#summary of newly created model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 1,281
Non-trainable params: 2,257,984
_________________________________________________________________


In [14]:
#training
model.fit_generator(train_generator,validation_data=validation_generator,steps_per_epoch=100,epochs=10,validation_steps=50,verbose=1)

Epoch 1/10
100/100 [==============================] - 137s 1s/step - loss: 0.5307 - acc: 0.7230 - val_loss: 0.4744 - val_acc: 0.7750
Epoch 2/10
100/100 [==============================] - 137s 1s/step - loss: 0.4837 - acc: 0.7700 - val_loss: 0.5089 - val_acc: 0.7600
Epoch 3/10
100/100 [==============================] - 135s 1s/step - loss: 0.4321 - acc: 0.8080 - val_loss: 0.4843 - val_acc: 0.7860
Epoch 4/10
100/100 [==============================] - 136s 1s/step - loss: 0.4096 - acc: 0.8115 - val_loss: 0.4307 - val_acc: 0.7980
Epoch 5/10
100/100 [==============================] - 135s 1s/step - loss: 0.3875 - acc: 0.8340 - val_loss: 0.4814 - val_acc: 0.7840
Epoch 6/10
100/100 [==============================] - 136s 1s/step - loss: 0.3717 - acc: 0.8270 - val_loss: 0.4669 - val_acc: 0.8060
Epoch 7/10
100/100 [==============================] - 136s 1s/step - loss: 0.3598 - acc: 0.8395 - val_loss: 0.4486 - val_acc: 0.8190
Epoch 8/10
100/100 [==============================] - 135s 1s/step - 

In [33]:
#fine tuning
base_model.trainable = True
#printing number of layers in model
print("number of layers:",len(base_model.layers))
fine_tune_at = 100
#freezing all the layers before fine_tune_at
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

number of layers: 155


In [0]:

model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=2e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [35]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 1,863,873
Non-trainable params: 395,392
_________________________________________________________________


In [36]:
len(model.trainable_variables)

58

In [0]:

model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=2e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [38]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 1,863,873
Non-trainable params: 395,392
_________________________________________________________________


In [41]:
history_fine = model.fit_generator(train_generator,
                                   steps_per_epoch = 100,
                                   epochs=10,
                                   workers=4,
                                   validation_data=validation_generator,
                                   validation_steps=50)

Epoch 1/10
100/100 [==============================] - 188s 2s/step - loss: 0.2727 - acc: 0.8830 - val_loss: 0.3528 - val_acc: 0.8700
Epoch 2/10
100/100 [==============================] - 182s 2s/step - loss: 0.2214 - acc: 0.9035 - val_loss: 0.2917 - val_acc: 0.8920
Epoch 3/10
100/100 [==============================] - 182s 2s/step - loss: 0.1995 - acc: 0.9105 - val_loss: 0.2887 - val_acc: 0.8950
Epoch 4/10
100/100 [==============================] - 183s 2s/step - loss: 0.1924 - acc: 0.9180 - val_loss: 0.2698 - val_acc: 0.8940
Epoch 5/10
100/100 [==============================] - 184s 2s/step - loss: 0.1817 - acc: 0.9235 - val_loss: 0.2543 - val_acc: 0.8980
Epoch 6/10
100/100 [==============================] - 183s 2s/step - loss: 0.1647 - acc: 0.9355 - val_loss: 0.2127 - val_acc: 0.9100
Epoch 7/10
100/100 [==============================] - 183s 2s/step - loss: 0.1484 - acc: 0.9415 - val_loss: 0.2144 - val_acc: 0.9140
Epoch 8/10
100/100 [==============================] - 183s 2s/step - 